### Ячейка 1: Импорты и глобальные настройки

In [1]:
# --- Системные и основные библиотеки ---
import os
import warnings
import pandas as pd
import numpy as np

# --- Визуализация ---
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams

# --- Машинное обучение (Scikit-learn) ---
from sklearn.model_selection import train_test_split

# --- Машинное обучение (PyTorch) ---
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

# --- Инструменты для экспериментов ---
import optuna
import mlflow
import mlflow.pytorch

# --- Игнорируем предупреждения для чистоты вывода ---
warnings.filterwarnings('ignore')

# --- Константы ---
DATA_PATH = "data"
FILE_TRAIN = os.path.join(DATA_PATH, "train.csv")
FILE_TEST = os.path.join(DATA_PATH, "test.csv")
SUBMISSION_FILE = 'submission_house_prices.csv'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Настройки для визуализации ---
plt.style.use('ggplot')
rcParams['figure.figsize'] = 12, 8

print("Все библиотеки импортированы.")
print(f"Обучение будет производиться на устройстве: {DEVICE}")
print(f"Optuna version: {optuna.__version__}")
print(f"MLflow version: {mlflow.__version__}")

Все библиотеки импортированы.
Обучение будет производиться на устройстве: cuda
Optuna version: 4.3.0
MLflow version: 3.1.0


### Ячейка 2: Конфигурация эксперимента

In [2]:
# --- Общие настройки эксперимента ---
EXPERIMENT_NAME = "House Prices - Optuna & MLflow"
N_TRIALS = 50 # Количество экспериментов для Optuna
EPOCHS = 200  # Фиксированное количество эпох для каждого эксперимента

# --- Базовые параметры (могут быть переопределены Optuna) ---
class Config:
    # Параметры DataLoader
    BATCH_SIZE = 128
    
    # Параметры оптимизатора
    LEARNING_RATE = 0.005
    WEIGHT_DECAY = 0.01
    
    # Параметры планировщика
    SCHEDULER_PATIENCE = 20
    SCHEDULER_FACTOR = 0.5
    
    # Параметры архитектуры модели
    # Список с количеством нейронов в каждом скрытом слое
    HIDDEN_LAYERS = [128, 64] 
    # Список со значением Dropout для каждого скрытого слоя
    DROPOUT_RATES = [0.4, 0.4]

print("Конфигурация эксперимента задана.")
print(f"Optuna проведет {N_TRIALS} экспериментов.")

Конфигурация эксперимента задана.
Optuna проведет 50 экспериментов.


### Ячейка 3: Комплексная обработка данных

In [3]:
# --- ЗАГРУЗКА ДАННЫХ ---
try:
    df_train = pd.read_csv(FILE_TRAIN)
    df_test = pd.read_csv(FILE_TEST)
    df_test_original = df_test.copy()
    print("Данные успешно загружены.")
except FileNotFoundError:
    print(f"Ошибка: Файлы не найдены в директории '{DATA_PATH}'.")

# --- ПОЛНЫЙ ПАЙПЛАЙН ПРЕДОБРАБОТКИ ---

# 1. Подготовка
df_train_proc = df_train.copy()
df_test_proc = df_test.copy()
train_target = df_train_proc['SalePrice'].copy()
df_train_proc.drop('SalePrice', axis=1, inplace=True)
train_rows = len(df_train_proc)
df_combined = pd.concat([df_train_proc, df_test_proc], ignore_index=True)


# --- ЭТАП 2: БАЗОВАЯ ОЧИСТКА И ЗАПОЛНЕНИЕ ПРОПУСКОВ ---

# 2.1 Рассчитываем медианы на ОБЪЕДИНЕННЫХ данных для большей точности
lotfrontage_median = df_combined['LotFrontage'].median()
garageyrblt_median = df_combined['GarageYrBlt'].median()

# 2.2 Заполняем пропуски
df_combined['LotFrontage'].fillna(lotfrontage_median, inplace=True)
df_combined['GarageYrBlt'].fillna(garageyrblt_median, inplace=True)

# 2.3 Заполняем нулями столбцы, где NA означает "отсутствие" объекта
cols_fill_zero = [
    'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
    'GarageCars', 'GarageArea', 'BsmtFullBath', 'BsmtHalfBath'
]
for col in cols_fill_zero:
    df_combined[col].fillna(0, inplace=True)

# 2.4 Заполняем пропуски в категориальных признаках строкой 'NA'
cat_cols_fill_na = [
    'Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
    'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
    'PoolQC', 'Fence', 'MiscFeature', 'MasVnrType', 'Electrical', 'MSZoning',
    'Utilities', 'Exterior1st', 'Exterior2nd', 'KitchenQual', 'Functional', 'SaleType'
]
for col in cat_cols_fill_na:
    df_combined[col].fillna('NA', inplace=True)

# 2.5 Исправляем опечатки, замеченные при анализе данных
df_combined['Exterior2nd'].replace({"CmentBd": "CemntBd", "Wd Shng": "WdShing"}, inplace=True)


# --- ЭТАП 3: ИНЖИНИРИНГ ПРИЗНАКОВ (FEATURE ENGINEERING) ---

# 3.1 Временные признаки (давность событий относительно 2010 года)
df_combined['YearBuildAgo'] = 2010 - df_combined['YearBuilt']
df_combined['YearRemodAddAgo'] = 2010 - df_combined['YearRemodAdd']
df_combined['GarageYrBltAgo'] = 2010 - df_combined['GarageYrBlt']
df_combined['MoSoldAgo'] = 12 - df_combined['MoSold'] + 12 * (2010 - df_combined['YrSold'])

# 3.2 Порядковое кодирование (Ordinal Encoding)
qual_map = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0}
bsmt_fin_map = {'GLQ': 5, 'ALQ': 4, 'BLQ': 3, 'Rec': 3.5, 'LwQ': 2, 'Unf': 1, 'NA': 0}
bsmt_exp_map = {'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'NA': 0}
garage_fin_map = {'Fin': 3, 'RFn': 2, 'Unf': 1, 'NA': 0}
functional_map = {'Typ': 0, 'Min1': 2, 'Min2': 1, 'Mod': 3, 'Maj1': 4, 'Maj2': 5, 'Sev': 6, 'NA': 0}
paved_drive_map = {'Y': 0, 'P': 1, 'N': 2}
electrical_map = {'SBrkr': 0, 'FuseA': 1, 'FuseF': 2, 'FuseP': 3, 'Mix': 1, 'NA': 1}

qual_cols = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']
for col in qual_cols:
    df_combined[col] = df_combined[col].map(qual_map)

df_combined['BsmtFinQuality1'] = df_combined['BsmtFinType1'].map(bsmt_fin_map)
df_combined['BsmtFinQuality2'] = df_combined['BsmtFinType2'].map(bsmt_fin_map)
df_combined['BsmtExposure'] = df_combined['BsmtExposure'].map(bsmt_exp_map)
df_combined['GarageFinish'] = df_combined['GarageFinish'].map(garage_fin_map)
df_combined['Functional'] = df_combined['Functional'].map(functional_map)
df_combined['PavedDrive'] = df_combined['PavedDrive'].map(paved_drive_map)
df_combined['Electrical'] = df_combined['Electrical'].map(electrical_map)

# 3.3 Создание признаков на основе условий
df_combined['Alley_road'] = df_combined['Alley'].map({'NA': 0, 'Grvl': 1, 'Pave': 2})
df_combined['LvlLotShape'] = df_combined['LotShape'].map({'Reg': 0, 'IR1': 1, 'IR2': 2, 'IR3': 3})
df_combined['LandContourLvl'] = df_combined['LandContour'].apply(lambda x: 0 if x == 'Lvl' else (1 if x == 'Low' else 2))
df_combined['Inside'] = (df_combined['LotConfig'] == 'Inside').astype(int)
df_combined['Corner'] = (df_combined['LotConfig'] == 'Corner').astype(int)
df_combined['CulDSac'] = (df_combined['LotConfig'] == 'CulDSac').astype(int)
df_combined['FR'] = df_combined['LotConfig'].apply(lambda x: 2 if x == 'FR3' else (1 if x == 'FR2' else 0))
df_combined['LandSlopeLvl'] = df_combined['LandSlope'].map({'Gtl': 0, 'Mod': 1, 'Sev': 2})
df_combined['RR'] = ((df_combined['Condition1'].str.contains('RR')) | (df_combined['Condition2'].str.contains('RR'))).astype(int)
df_combined['PosObj'] = ((df_combined['Condition1'].str.contains('Pos')) | (df_combined['Condition2'].str.contains('Pos'))).astype(int)
df_combined['StreetObj'] = ((df_combined['Condition1'].isin(['Artery', 'Feedr'])) | (df_combined['Condition2'].isin(['Artery', 'Feedr']))).astype(int)
df_combined['notFinishFloor'] = ((df_combined['HouseStyle'] == '1.5Unf') | (df_combined['HouseStyle'] == '2.5Unf')).astype(int)
df_combined['Split'] = df_combined['HouseStyle'].apply(lambda x: 2 if x == 'SLvl' else (1 if x == 'SFoyer' else 0))
df_combined['NumGarage'] = df_combined.apply(lambda row: 2 if row['GarageType'] == '2Types' or row['MiscFeature'] == 'Gar2' else 0 if row['GarageType'] == 'NA' else 1, axis=1)
df_combined['Privacy'] = df_combined['Fence'].apply(lambda x: 2 if x == 'GdPrv' else (1 if x in ['MnPrv', 'GdWo'] else 0))
df_combined['WoodFence'] = df_combined['Fence'].apply(lambda x: 2 if x == 'GdWo' else (1 if x == 'MnWw' else 0))
df_combined['Is_Rec_Room1'] = (df_combined['BsmtFinType1'] == 'Rec').astype(int)
df_combined['Is_Rec_Room2'] = (df_combined['BsmtFinType2'] == 'Rec').astype(int)

# 3.4 Группировка редких категорий и упрощение признаков
df_combined['Neighborhood'] = df_combined['Neighborhood'].apply(lambda x: 'Other' if x in ['Veenker', 'NPkVill', 'Blmngtn', 'Blueste'] else x)
df_combined['MSZoning'] = df_combined['MSZoning'].apply(lambda x: 'Other' if x in ['C (all)', 'NA', 'RH'] else x)
df_combined['BldgType'] = df_combined['BldgType'].apply(lambda x: 'Other' if x in ['Duplex', 'Twnhs', '2fmCon'] else x)
df_combined['RoofStyle'] = df_combined['RoofStyle'].apply(lambda x: 'Other' if x in ['Gambrel', 'Flat', 'Mansard', 'Shed'] else x)
df_combined['RoofMatl'] = df_combined['RoofMatl'].apply(lambda x: 'Other' if x in ["Tar&Grv", "ClyTile", "Membran", "Metal", "Roll", "WdShngl", "WdShake"] else x)
df_combined['MasVnrType'] = df_combined['MasVnrType'].apply(lambda x: 'Other' if x in ['NA', 'BrkCmn'] else x)
df_combined['Foundation'] = df_combined['Foundation'].apply(lambda x: 'Other' if x in ['Stone', 'Wood'] else x)
df_combined['Heating'] = df_combined['Heating'].apply(lambda x: 'Other' if x in ['GasW', 'Grav', 'Wall', 'OthW', 'Floor'] else x)
df_combined['GarageType'] = df_combined['GarageType'].apply(lambda x: 'Other' if x in ['Basment', '2Types', 'CarPort'] else x)
df_combined['MiscFeature'] = df_combined['MiscFeature'].apply(lambda x: 'ShedOrOther' if x in ['Gar2', 'Othr', 'TenC', 'Shed'] else x)
df_combined['SaleType'] = df_combined['SaleType'].apply(lambda x: 'Other' if x in ['ConLD', 'CWD', 'ConLI', 'ConLw', 'Oth', 'Con', 'NA'] else x)
df_combined['SaleCondition'] = df_combined['SaleCondition'].apply(lambda x: 'Other' if x in ['Alloca', 'AdjLand'] else x)

# 3.5 One-Hot Encoding для материалов экстерьера
main_materials = ["VinylSd", "MetalSd", "HdBoard", "Wd Sdng", "Plywood", "CemntBd", "BrkFace", "WdShing", "AsbShng", "Stucco"]
for material in main_materials:
    df_combined[material] = ((df_combined['Exterior1st'] == material) | (df_combined['Exterior2nd'] == material)).astype(int)
df_combined['ExtMat_Other'] = ((~df_combined['Exterior1st'].isin(main_materials)) | (~df_combined['Exterior2nd'].isin(main_materials))).astype(int)


# --- ЭТАП 4: ФИНАЛИЗАЦИЯ И РАЗДЕЛЕНИЕ ---

# 4.1 Удаляем исходные столбцы, которые были преобразованы или больше не нужны
cols_to_drop = [
    'Id', 'MSSubClass', 'TotalBsmtSF', 'Street', 'Utilities', 'Alley', 'LotShape',
    'LandContour', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType',
    'HouseStyle', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'Exterior1st', 'Exterior2nd',
    'BsmtFinType1', 'BsmtFinType2', 'GarageYrBlt', 'Fence', 'YrSold', 'MoSold'
]
df_processed = df_combined.drop(columns=cols_to_drop, errors='ignore')

# 4.2 Преобразуем оставшиеся категориальные столбцы в dummy-переменные
df_processed = pd.get_dummies(df_processed, drop_first=True)

# 4.3 Преобразуем столбцы типа 'bool' (True/False) в 'int' (1/0)
bool_columns = df_processed.select_dtypes(include=['bool']).columns
df_processed[bool_columns] = df_processed[bool_columns].astype(int)


# --- ЭТАП 5: РАЗДЕЛЕНИЕ НА TRAIN И TEST ---

# 5.1 Разделяем обработанный датафрейм обратно на train и test
df_train_proc = df_processed.iloc[:train_rows].copy()
df_test_proc = df_processed.iloc[train_rows:].copy()

# --- КОНЕЦ СКРИПТА ---
print("Предобработка завершена.")
print(f"Размер обработанного train датасета: {df_train_proc.shape}")
print(f"Размер обработанного test датасета:  {df_test_proc.shape}")
print(f"Целевая переменная 'SalePrice' сохранена отдельно, размер: {train_target.shape}")

Данные успешно загружены.
Предобработка завершена.
Размер обработанного train датасета: (1460, 124)
Размер обработанного test датасета:  (1459, 124)
Целевая переменная 'SalePrice' сохранена отдельно, размер: (1460,)


### Ячейка 4: Подготовка данных для обучения


In [4]:
X = df_train_proc
y = train_target

# Логарифмируем целевую переменную для стабилизации обучения
y_log = np.log1p(y)

# Разделение на обучающую и валидационную выборки
X_train, X_val, y_train_log, y_val_log = train_test_split(
    X, y_log, 
    test_size=0.2, 
    random_state=42 # для воспроизводимости результатов
)

print("Данные разделены на обучающую и валидационную выборки.")
print(f"X_train: {X_train.shape}, y_train: {y_train_log.shape}")
print(f"X_val:   {X_val.shape}, y_val:   {y_val_log.shape}")

Данные разделены на обучающую и валидационную выборки.
X_train: (1168, 124), y_train: (1168,)
X_val:   (292, 124), y_val:   (292,)


### Ячейка 5: Класс-обертка для данных (PyTorch Dataset)

In [5]:
# Преобразуем данные pandas в формат тензоров PyTorch
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_log.values, dtype=torch.float32).unsqueeze(1)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val_log.values, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(df_test_proc.values, dtype=torch.float32)

# Создание кастомного Dataset
class HousesDataset(Dataset):
    """
    Класс-обертка для данных о домах.
    Позволяет DataLoader'у эффективно работать с нашими тензорами.
    """
    def __init__(self, features, labels=None):
        self.features = features
        self.labels = labels
        # Если метки (labels) не переданы, значит, это тестовый набор
        self.is_test = labels is None
    
    def __len__(self):
        # Возвращает общее количество примеров в датасете
        return len(self.features)
    
    def __getitem__(self, idx):
        # Возвращает один пример (признаки и, если есть, метку) по индексу
        if self.is_test:
            return self.features[idx]
        else:
            return self.features[idx], self.labels[idx]

print("Класс HousesDataset и тензоры данных готовы к использованию.")

Класс HousesDataset и тензоры данных готовы к использованию.


### Ячейка 6: Динамическая архитектура модели

In [6]:
class DynamicRegressionModel(nn.Module):
    """
    Нейронная сеть, архитектура которой определяется входными параметрами.
    """
    def __init__(self, num_features, hidden_layers, dropout_rates):
        super().__init__()
        
        layers = []
        input_dim = num_features
        
        # Динамически создаем скрытые слои
        for i, (hidden_dim, dropout_rate) in enumerate(zip(hidden_layers, dropout_rates)):
            layers.append(nn.BatchNorm1d(input_dim))
            layers.append(nn.Linear(input_dim, hidden_dim))
            layers.append(nn.GELU())
            layers.append(nn.Dropout(dropout_rate))
            input_dim = hidden_dim # Выход текущего слоя - вход для следующего
            
        # Добавляем выходной слой
        layers.append(nn.BatchNorm1d(input_dim))
        layers.append(nn.Linear(input_dim, 1))
        
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# Пример создания модели с базовой конфигурацией
input_features = X_train.shape[1]
example_model = DynamicRegressionModel(
    num_features=input_features,
    hidden_layers=Config.HIDDEN_LAYERS,
    dropout_rates=Config.DROPOUT_RATES
).to(DEVICE)

print("Архитектура модели стала динамической. Пример:")
print(example_model)

Архитектура модели стала динамической. Пример:
DynamicRegressionModel(
  (network): Sequential(
    (0): BatchNorm1d(124, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=124, out_features=128, bias=True)
    (2): GELU(approximate='none')
    (3): Dropout(p=0.4, inplace=False)
    (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Linear(in_features=128, out_features=64, bias=True)
    (6): GELU(approximate='none')
    (7): Dropout(p=0.4, inplace=False)
    (8): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Linear(in_features=64, out_features=1, bias=True)
  )
)


### Ячейка 7: objective — Сердце эксперимента

In [7]:
def objective(trial):
    """
    Функция для одного эксперимента Optuna. 
    Обучает модель с предложенными гиперпараметрами и возвращает метрику качества.
    """
    # Запускаем новый "прогон" в MLflow. `nested=True` нужно для Optuna.
    with mlflow.start_run(nested=True):
        
        # --- 1. Предложение гиперпараметров от Optuna ---
        # Мы создаем словарь, чтобы потом удобно логировать его в MLflow
        params = {
            'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True),
            'batch_size': trial.suggest_categorical('batch_size', [64, 128, 256]),
            'weight_decay': trial.suggest_float('weight_decay', 1e-5, 1e-1, log=True),
            'num_hidden_layers': trial.suggest_int('num_hidden_layers', 1, 3),
        }
        
        hidden_layers = []
        dropout_rates = []
        for i in range(params['num_hidden_layers']):
            # Используем log=True для n_units, т.к. это широкий диапазон
            hidden_layers.append(trial.suggest_int(f'n_units_l{i}', 32, 256, log=True))
            dropout_rates.append(trial.suggest_float(f'dropout_l{i}', 0.1, 0.5))
        
        # Логируем все параметры разом
        mlflow.log_params(trial.params)
        # Логируем архитектуру в читаемом виде
        mlflow.log_param("hidden_layers_arch", str(hidden_layers))
        mlflow.log_param("dropout_rates_arch", str(dropout_rates))

        # --- 2. Подготовка данных и модели ---
        train_loader = DataLoader(HousesDataset(X_train_tensor, y_train_tensor), batch_size=params['batch_size'], shuffle=True)
        val_loader = DataLoader(HousesDataset(X_val_tensor, y_val_tensor), batch_size=params['batch_size'], shuffle=False)

        model = DynamicRegressionModel(
            X_train.shape[1], 
            hidden_layers, 
            dropout_rates
        ).to(DEVICE)
        
        optimizer = torch.optim.RAdam(model.parameters(), lr=params['learning_rate'], weight_decay=params['weight_decay'])
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=Config.SCHEDULER_FACTOR, patience=Config.SCHEDULER_PATIENCE)
        criterion = nn.MSELoss()

        # --- 3. Цикл обучения и валидации ---
        min_val_loss = float('inf')
        for epoch in range(EPOCHS):
            model.train()
            # ... (цикл обучения без изменений) ...
            for features, labels in train_loader:
                features, labels = features.to(DEVICE), labels.to(DEVICE)
                optimizer.zero_grad()
                outputs = model(features)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            model.eval()
            current_val_loss = 0.0
            with torch.no_grad():
                for features, labels in val_loader:
                    features, labels = features.to(DEVICE), labels.to(DEVICE)
                    outputs = model(features)
                    loss = criterion(outputs, labels)
                    current_val_loss += loss.item() * features.size(0)
            
            epoch_val_loss = np.sqrt(current_val_loss / len(val_loader.dataset))
            scheduler.step(epoch_val_loss)
            
            mlflow.log_metric('val_rmse', epoch_val_loss, step=epoch)

            if epoch_val_loss < min_val_loss:
                min_val_loss = epoch_val_loss
        
        # --- 4. Финальное логирование в MLflow ---
        mlflow.log_metric('final_val_rmse', min_val_loss)
        
        # ИЗМЕНЕНИЕ 1: Создаем пример входных данных для сигнатуры
        input_example = X_train_tensor[:5].to(DEVICE) # Берем 5 примеров
        
        # ИЗМЕНЕНИЕ 2: Логируем модель с сигнатурой и регистрируем ее
        mlflow.pytorch.log_model(
            model, 
            "model", # Имя папки с артефактом
            signature=mlflow.models.infer_signature(input_example.cpu().numpy(), model(input_example).detach().cpu().numpy()),
            input_example=input_example.cpu().numpy(),
            registered_model_name="house-prices-regressor" # Имя для реестра моделей
        )

    return min_val_loss

### Ячейка 8: Запуск оптимизации

In [8]:
# Устанавливаем имя эксперимента в MLflow
mlflow.set_experiment(EXPERIMENT_NAME)

# Создаем "исследование" Optuna
study = optuna.create_study(direction='minimize')

# Запускаем оптимизацию
# Optuna будет вызывать objective N_TRIALS раз
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True)

# --- Вывод результатов ---
print("\nОптимизация завершена!")
print(f"Количество завершенных экспериментов: {len(study.trials)}")
print("Лучший эксперимент:")
best_trial = study.best_trial
print(f"  Значение (min val_rmse): {best_trial.value:.4f}")
print("  Лучшие гиперпараметры: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

[I 2025-06-12 18:35:18,587] A new study created in memory with name: no-name-4e989fa9-0b74-4b64-a962-26c2f0f3728d


  0%|          | 0/50 [00:00<?, ?it/s]

2025/06/12 18:35:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/12 18:35:37 WARNING mlflow.utils.requirements_utils: Found torch version (2.8.0a0+5228986c39.nv25.5) contains a local version label (+5228986c39.nv25.5). MLflow logged a pip requirement for this package as 'torch==2.8.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/06/12 18:35:42 WARNING mlflow.utils.requirements_utils: Found torch version (2.8.0a0+5228986c39.nv25.5) contains a local version label (+5228986c39.nv25.5). MLflow logged a pip requirement for this package as 'torch==2.8.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
Registered model 'house-prices-regressor' already exists. Creating a new version of t

[I 2025-06-12 18:35:43,363] Trial 0 finished with value: 0.26429730003397245 and parameters: {'learning_rate': 0.00016167997948068327, 'batch_size': 64, 'weight_decay': 0.00025311386299310787, 'num_hidden_layers': 3, 'n_units_l0': 159, 'dropout_l0': 0.1936479896012706, 'n_units_l1': 234, 'dropout_l1': 0.39566740072844786, 'n_units_l2': 135, 'dropout_l2': 0.11371163278341499}. Best is trial 0 with value: 0.26429730003397245.


2025/06/12 18:35:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/12 18:35:52 WARNING mlflow.utils.requirements_utils: Found torch version (2.8.0a0+5228986c39.nv25.5) contains a local version label (+5228986c39.nv25.5). MLflow logged a pip requirement for this package as 'torch==2.8.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/06/12 18:35:55 WARNING mlflow.utils.requirements_utils: Found torch version (2.8.0a0+5228986c39.nv25.5) contains a local version label (+5228986c39.nv25.5). MLflow logged a pip requirement for this package as 'torch==2.8.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
Registered model 'house-prices-regressor' already exists. Creating a new version of t

...

[I 2025-06-12 18:45:12,470] Trial 47 finished with value: 0.12676121101096596 and parameters: {'learning_rate': 0.004397932188727885, 'batch_size': 128, 'weight_decay': 0.007404953432759068, 'num_hidden_layers': 2, 'n_units_l0': 159, 'dropout_l0': 0.35474580388780685, 'n_units_l1': 112, 'dropout_l1': 0.14422543828537043}. Best is trial 41 with value: 0.12666372253541197.


2025/06/12 18:45:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/12 18:45:26 WARNING mlflow.utils.requirements_utils: Found torch version (2.8.0a0+5228986c39.nv25.5) contains a local version label (+5228986c39.nv25.5). MLflow logged a pip requirement for this package as 'torch==2.8.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/06/12 18:45:30 WARNING mlflow.utils.requirements_utils: Found torch version (2.8.0a0+5228986c39.nv25.5) contains a local version label (+5228986c39.nv25.5). MLflow logged a pip requirement for this package as 'torch==2.8.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
Registered model 'house-prices-regressor' already exists. Creating a new version of t

[I 2025-06-12 18:45:31,042] Trial 48 finished with value: 0.1307833560346617 and parameters: {'learning_rate': 0.006929193391242593, 'batch_size': 64, 'weight_decay': 0.0010689935110940424, 'num_hidden_layers': 2, 'n_units_l0': 161, 'dropout_l0': 0.3620066425237005, 'n_units_l1': 126, 'dropout_l1': 0.10090645942587279}. Best is trial 41 with value: 0.12666372253541197.


2025/06/12 18:45:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/12 18:45:38 WARNING mlflow.utils.requirements_utils: Found torch version (2.8.0a0+5228986c39.nv25.5) contains a local version label (+5228986c39.nv25.5). MLflow logged a pip requirement for this package as 'torch==2.8.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/06/12 18:45:42 WARNING mlflow.utils.requirements_utils: Found torch version (2.8.0a0+5228986c39.nv25.5) contains a local version label (+5228986c39.nv25.5). MLflow logged a pip requirement for this package as 'torch==2.8.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
Registered model 'house-prices-regressor' already exists. Creating a new version of t

[I 2025-06-12 18:45:42,905] Trial 49 finished with value: 0.24153830604751228 and parameters: {'learning_rate': 0.0007677923666755756, 'batch_size': 128, 'weight_decay': 0.002816463149629236, 'num_hidden_layers': 2, 'n_units_l0': 45, 'dropout_l0': 0.3931512125718742, 'n_units_l1': 99, 'dropout_l1': 0.13756574408861727}. Best is trial 41 with value: 0.12666372253541197.

Оптимизация завершена!
Количество завершенных экспериментов: 50
Лучший эксперимент:
  Значение (min val_rmse): 0.1267
  Лучшие гиперпараметры: 
    learning_rate: 0.005990872975138827
    batch_size: 128
    weight_decay: 0.004876583689765625
    num_hidden_layers: 2
    n_units_l0: 79
    dropout_l0: 0.29342350087894237
    n_units_l1: 32
    dropout_l1: 0.2090323496968174



### Ячейка 9: Обучение финальной модели и создание Submission

In [9]:
print("Обучение финальной модели на лучших параметрах...")

# --- 1. Получаем лучшие параметры ---
best_params = study.best_trial.params
# MLflow автоматически находит лучший run в текущем эксперименте
best_run = mlflow.search_runs(order_by=['metrics.final_val_rmse ASC']).iloc[0]

print(f"Загрузка модели из MLflow run_id: {best_run.run_id}")

# --- 2. Загружаем лучшую модель напрямую из MLflow ---
# Это более надежно, чем переобучать заново
best_model_uri = f"runs:/{best_run.run_id}/model"
final_model = mlflow.pytorch.load_model(best_model_uri).to(DEVICE)

# --- 3. Создание предсказаний ---
final_model.eval()
X_test_tensor = torch.tensor(df_test_proc.values, dtype=torch.float32)
test_loader = DataLoader(HousesDataset(X_test_tensor), batch_size=256, shuffle=False)

all_predictions = []
with torch.no_grad():
    for features_batch in test_loader:
        features_batch = features_batch.to(DEVICE)
        outputs = final_model(features_batch)
        predicted_prices = torch.expm1(outputs)
        all_predictions.extend(predicted_prices.cpu().numpy().flatten().tolist())

# --- 4. Создание файла для отправки ---
submission_df = pd.DataFrame({
    'Id': df_test_original['Id'],
    'SalePrice': all_predictions
})
submission_df.to_csv(SUBMISSION_FILE, index=False)

print(f"\nSubmission файл '{SUBMISSION_FILE}' успешно создан.")
print("Первые 5 строк submission файла:")
print(submission_df.head())

Обучение финальной модели на лучших параметрах...
Загрузка модели из MLflow run_id: 4e08d207ecb24f5d95ab46fe2520bd7d



Submission файл 'submission_house_prices.csv' успешно создан.
Первые 5 строк submission файла:
     Id      SalePrice
0  1461  114508.132812
1  1462  158595.906250
2  1463  177283.859375
3  1464  191931.093750
4  1465  191447.203125
